In [1]:
# Import useful packages
import os
import time
import pandas as pd
import glob
import numpy as np

# For loading/querying postgres
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import connect

In [2]:
# Create SQL engine to load data to postgres for central usage
from sqlalchemy import create_engine
dbname = '697_temp'
user = 'postgres'
password = 'poRter!5067'
host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com'

engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

### Loading Open Positions to Postgres

In [ ]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Open_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

In [ ]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

In [ ]:
# Initial dataframe needs to create a new table
temp_df.to_sql('open_positions', engine)

In [ ]:
# Test with a load
def import_open_positions():
#     conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM open_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['key', 'model'])
    return data
open_pos_df = import_open_positions().drop('index',axis=1)

open_pos_df.tail()


### Loading track records to postgres

In [ ]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("track_record/*.csv"):
    txtfiles.append(file)
    
txtfiles

In [ ]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.head()

In [ ]:
# Initial dataframe needs to create a new table
temp_df.to_sql('track_record', engine)

In [16]:
# Test with a load
def import_track_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM track_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['Date', 'model'])
    return data
track_df = import_track_record().drop('index',axis=1)

track_df.head()


,Date,Value,Val_ex_cash,model
0,2020-08-06,100000.000000,99591.157106,RF Reg_target_120_rebal_30_2017-01-01
1,2020-08-07,100404.519099,99995.676205,RF Reg_target_120_rebal_30_2017-01-01
2,2020-08-10,104338.074718,103929.231824,RF Reg_target_120_rebal_30_2017-01-01
3,2020-08-11,108502.006702,108093.163809,RF Reg_target_120_rebal_30_2017-01-01
4,2020-08-12,106806.659895,106397.817002,RF Reg_target_120_rebal_30_2017-01-01


### Loading cash records to postgres

In [4]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Cash_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Cash_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv']

In [5]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.tail()

,key,cash,model
139,cash_2021-02-25,22.961293,RF Reg_target_120_rebal_30_2017-01-01
140,cash_2021-02-26,22.961293,RF Reg_target_120_rebal_30_2017-01-01
141,cash_2021-03-01,22.961293,RF Reg_target_120_rebal_30_2017-01-01
142,cash_2021-03-02,22.961293,RF Reg_target_120_rebal_30_2017-01-01
143,cash_2021-03-03,22.961293,RF Reg_target_120_rebal_30_2017-01-01


In [13]:
# Initial dataframe needs to create a new table
temp_df.to_sql('cash_record', engine,if_exists='append')

In [14]:
# Test with a load
def import_cash_record():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM cash_record"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
cash_df = import_cash_record().drop('index',axis=1)

cash_df.tail()


,key,cash,model
139,cash_2021-02-25,22.961293,RF Reg_target_120_rebal_30_2017-01-01
140,cash_2021-02-26,22.961293,RF Reg_target_120_rebal_30_2017-01-01
141,cash_2021-03-01,22.961293,RF Reg_target_120_rebal_30_2017-01-01
142,cash_2021-03-02,22.961293,RF Reg_target_120_rebal_30_2017-01-01
143,cash_2021-03-03,22.961293,RF Reg_target_120_rebal_30_2017-01-01


In [15]:
cash_df.count()

key      144
cash     144
model    144
dtype: int64

### Loading Value Positions to postgres

In [17]:
# Collect all CSVs in Open_Positions_Data folder
txtfiles = []
for file in glob.glob("Val_Positions_Data/*.csv"):
    txtfiles.append(file)
    
txtfiles

['Val_Positions_Data/RF Reg_target_120_rebal_30_2017-01-01.csv']

In [18]:
# create temp_df to test loading
temp_df = pd.read_csv(txtfiles[0],index_col=0)
temp_df.tail()

,key,value,model
139,val_2021-02-25,169358.899298,RF Reg_target_120_rebal_30_2017-01-01
140,val_2021-02-26,163775.908318,RF Reg_target_120_rebal_30_2017-01-01
141,val_2021-03-01,167727.639779,RF Reg_target_120_rebal_30_2017-01-01
142,val_2021-03-02,167756.624743,RF Reg_target_120_rebal_30_2017-01-01
143,val_2021-03-03,171210.687314,RF Reg_target_120_rebal_30_2017-01-01


In [19]:
# Initial dataframe needs to create a new table
temp_df.to_sql('val_positions', engine,if_exists='append')

In [21]:
# Test with a load
def import_val_positions():
    conn = connect(dbname = '697_temp', user = 'postgres', host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com', password = 'poRter!5067')
    cur = conn.cursor()
    query = "SELECT * FROM val_positions"
    data = pd.read_sql_query(query,conn)
    data = data.sort_values(['model','key'])
    return data
val_df = import_val_positions().drop('index',axis=1)

val_df.tail()

,key,value,model
139,val_2021-02-25,169358.899298,RF Reg_target_120_rebal_30_2017-01-01
140,val_2021-02-26,163775.908318,RF Reg_target_120_rebal_30_2017-01-01
141,val_2021-03-01,167727.639779,RF Reg_target_120_rebal_30_2017-01-01
142,val_2021-03-02,167756.624743,RF Reg_target_120_rebal_30_2017-01-01
143,val_2021-03-03,171210.687314,RF Reg_target_120_rebal_30_2017-01-01
